# Introduction

Implement YOLO usage as explained in the [reference article](https://towardsdatascience.com/yolo-object-detection-with-opencv-and-python-21e50ac599e9).

<br>

The model has been trained over the [COCO Dataset](https://cocodataset.org/#home).

In [2]:
# Import Standard Libraries
import cv2

In [3]:
# Notebook variables
sample_image_path = './../images/dog_image_1.jpeg'
classes_path = './../files/yolov3_classes.txt'

# Read Data

## Read Image

In [5]:
# Read the image with OpenCV
image = cv2.imread(sample_image_path)

In [9]:
# The image is 576 x 768 dimension in pixels
# It has 3 channels, since it is colored (RGB)
image.shape

(576, 768, 3)

In [13]:
# Compute image's width and height
image_width = image.shape[1]
image_height = image.shape[0]
scale = 0.00392

 ## Read Classes

In [14]:
# Open the classes file and extract the list of available classes
with open(classes_path, 'r') as classes_file:
    classes = [line.strip() for line in classes_file.readlines()]